In [1]:
from pathlib import Path

ROOT = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset")
SRC  = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset\tumveriler")

print("ROOT exists:", ROOT.exists())
print("SRC exists :", SRC.exists())


ROOT exists: True
SRC exists : True


In [2]:
from pathlib import Path

IMG_EXT = {".jpg",".jpeg",".png",".bmp",".webp"}
SRC = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset\tumveriler")

imgs = [p for p in SRC.rglob("*") if p.suffix.lower() in IMG_EXT]
labels = [p for p in SRC.rglob("*.txt")]

img_stems = {p.stem for p in imgs}
lab_stems = {p.stem for p in labels}

only_imgs = sorted(list(img_stems - lab_stems))
only_labs = sorted(list(lab_stems - img_stems))

print("Toplam görsel:", len(imgs))
print("Toplam label :", len(labels))
print("Label'sız görsel (stem):", len(only_imgs))
print("Görselsiz label (stem):", len(only_labs))

if len(only_imgs) > 0:
    print("Örnek label'sız görsel:", only_imgs[:5])
if len(only_labs) > 0:
    print("Örnek görselsiz label:", only_labs[:5])


Toplam görsel: 1261
Toplam label : 1262
Label'sız görsel (stem): 0
Görselsiz label (stem): 1
Örnek görselsiz label: ['classes']


In [6]:
import random, shutil
from pathlib import Path

IMG_EXT = {".jpg",".jpeg",".png",".bmp",".webp"}

SRC = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset\tumveriler")
OUT = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset\dataset_yolo8")

# klasörler
for split in ["train","val","test"]:
    (OUT/split/"images").mkdir(parents=True, exist_ok=True)
    (OUT/split/"labels").mkdir(parents=True, exist_ok=True)

# eşleşen (etiketli) görselleri al
imgs = [p for p in SRC.rglob("*") if p.suffix.lower() in IMG_EXT]
labels = {p.stem: p for p in SRC.rglob("*.txt")}
paired = [p for p in imgs if p.stem in labels]

print("Eşleşen etiketli görüntü:", len(paired))

# split
seed = 42
random.seed(seed)
paired = sorted(paired, key=lambda p: p.name)
random.shuffle(paired)

n = len(paired)
n_train = int(n * 0.80)
n_val   = int(n * 0.10)
n_test  = n - n_train - n_val

train_files = paired[:n_train]
val_files   = paired[n_train:n_train+n_val]
test_files  = paired[n_train+n_val:]

def copy_split(files, split_name):
    for img in files:
        lab = labels[img.stem]
        shutil.copy2(img, OUT/split_name/"images"/img.name)
        shutil.copy2(lab, OUT/split_name/"labels"/lab.name)

copy_split(train_files, "train")
copy_split(val_files, "val")
copy_split(test_files, "test")

print("TRAIN:", len(train_files))
print("VAL  :", len(val_files))
print("TEST :", len(test_files))
print("Toplam:", len(train_files)+len(val_files)+len(test_files))
print("OUT:", OUT)


Eşleşen etiketli görüntü: 1261
TRAIN: 1008
VAL  : 126
TEST : 127
Toplam: 1261
OUT: C:\Users\AhmetYasinUgur\Desktop\dataset\dataset_yolo8


In [7]:
import yaml
from pathlib import Path

DATA_YAML = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset\data.yaml")
OUT = Path(r"C:\Users\AhmetYasinUgur\Desktop\dataset\dataset_yolo8")

data = {
    "path": str(OUT),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "names": ["acilcikis", "yayagecidi", "dur", "hizgosterge"]
}

DATA_YAML.write_text(yaml.safe_dump(data, sort_keys=False, allow_unicode=True), encoding="utf-8")
print("Yazıldı:", DATA_YAML)
print(DATA_YAML.read_text(encoding="utf-8"))


Yazıldı: C:\Users\AhmetYasinUgur\Desktop\dataset\data.yaml
path: C:\Users\AhmetYasinUgur\Desktop\dataset\dataset_yolo8
train: train/images
val: val/images
test: test/images
names:
- acilcikis
- yayagecidi
- dur
- hizgosterge



In [8]:
from ultralytics import YOLO

DATA_YAML = r"C:\Users\AhmetYasinUgur\Desktop\dataset\data.yaml"

model = YOLO("yolov8n.pt")

results = model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,      # GPU
    workers=0,
    optimizer="AdamW",
    project=r"C:\Users\AhmetYasinUgur\Desktop\dataset\runs_yolov8",
    name="exp1"
)


New https://pypi.org/project/ultralytics/8.3.241 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.240  Python-3.11.1 torch-2.11.0.dev20251223+cu128 CUDA:0 (NVIDIA GeForce RTX 5050 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\AhmetYasinUgur\Desktop\dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_s

In [10]:
!pip install pandas


In [2]:
from ultralytics import YOLO

model = YOLO(r"C:\Users\AhmetYasinUgur\Desktop\dataset\runs_yolov8\exp1\weights\best.pt")
data_yaml = r"C:\Users\AhmetYasinUgur\Desktop\dataset\data.yaml"

confs = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.50, 0.60]

best = {"conf": None, "F1": 0}

for c in confs:
    m = model.val(
        data=data_yaml,
        split="val",
        conf=c,
        iou=0.5,
        plots=False,
        verbose=False
    )
    P = float(m.box.mp)
    R = float(m.box.mr)
    F1 = (2*P*R/(P+R)) if (P+R) > 0 else 0.0

    print(f"conf={c:.2f} | P={P:.3f} R={R:.3f} F1={F1:.3f}")

    if F1 > best["F1"]:
        best = {"conf": c, "F1": F1}

print("\n✅ EN İYİ CONF:", best)


Ultralytics 8.3.240  Python-3.11.1 torch-2.11.0.dev20251223+cu128 CUDA:0 (NVIDIA GeForce RTX 5050 Laptop GPU, 8151MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 228.7332.0 MB/s, size: 278.0 KB)
val: Scanning C:\Users\AhmetYasinUgur\Desktop\dataset\dataset_yolo8\val\labels.cache... 126 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 126/126  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.2it/s 6.5s0.2ss
                   all        126        126      0.929      0.917      0.949      0.649
Speed: 1.8ms preprocess, 6.0ms inference, 0.0ms loss, 2.4ms postprocess per image
conf=0.05 | P=0.929 R=0.917 F1=0.923
Ultralytics 8.3.240  Python-3.11.1 torch-2.11.0.dev20251223+cu128 CUDA:0 (NVIDIA GeForce RTX 5050 Laptop GPU, 8151MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 228.081.7 MB/s, size: 50.4 KB)
val: Scanning C:\Users\Ahm

In [3]:
import sys
!{sys.executable} -m pip install -U pandas


   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.3 MB 1.5 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/11.3 MB 1.8 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/11.3 MB 2.0 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.3 MB 2.2 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.3 MB 2.4 MB/s eta 0:00:04
   ------------ --------------------------- 3.7/11.3 MB 2.4 MB/s eta 0:00:04
   ------------- -------------------------- 3.9/11.3 MB 2.4 MB/s eta 0:00:04
   -------------- -----------------------

In [4]:
import pandas as pd
pd.__version__


'2.3.3'

In [5]:
SAVE_ROOT = r"C:\Users\AhmetYasinUgur\Desktop\dataset\runs\detect"
